In [211]:
import pandas as pd
import numpy as np
import random as rnd
import re

import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
train_df = pd.read_csv('../data_train.csv', low_memory=False, index_col=0)
test_df = pd.read_csv('../data_test.csv', low_memory=False, index_col=0)

In [7]:
print(train_df.shape, test_df.shape)
print(train_df.describe().shape, test_df.describe().shape)

# unique values of each feature
# for i in range(0,10):
#     print(i, train_df.iloc[:, i].unique().size)
# # print(train_df.iloc[:, i].dtype)

((38199, 262), (9538, 262))
((8, 18), (8, 43))


### numerical and null propotion of each feature

In [168]:
def isFloat(aString):
    try:
        float(aString)
        return True
    except:
        return False

isnumerical_df = train_df.applymap(lambda x: isFloat(x))
numerical_prop = (isnumerical_df.sum() - train_df.isnull().sum())/(train_df.shape[0] - train_df.isnull().sum())
null_prop = train_df.isnull().sum() / train_df.shape[0]

0     136.00
1     188.00
2        176
3        137
4        144
5        162
6        NaN
7     163.00
8        NaN
9        NaN
10       NaN
Name: 313, dtype: object


### select numerical features by numerical and null propotion

In [197]:
numeric_features = []
for i in range(train_df.shape[1]):
    if numerical_prop[i] > 0.8 and null_prop[i] < 0.8:
        numeric_features.append(i)
print('number of numeric features: ' + str(len(numeric_features)))

number of numeric features: 75


**look at a specific feature**

In [262]:
i = 13
print('featue id: %s, numerical proportion: %.5f, nan proportion: %.5f' % 
      (train_df.columns[numeric_features[i]], numerical_prop[numeric_features[i]], null_prop[numeric_features[i]]))
print('unique values:', train_df.iloc[:, numeric_features[i]].unique())

featue id: 1117, numerical proportion: 0.99877, nan proportion: 0.31713
('unique values:', array([  23.  ,  162.  ,  115.  , ...,  201.4 ,   43.28,   46.09]))


In [263]:
def toFloat(aString):
    try:
        result = float(aString)
        return result
    except:
        gro = re.search(r'(-?\d+\.?\d+)', aString)
        if gro:
            return float(gro.group(0))
        else:
            return float('NaN')


In [275]:
# update train/test set
train_df_new = train_df
for feature_idx in numeric_features[5:]:
    train_df_new.iloc[:, feature_idx] = train_df.iloc[:, feature_idx].map(toFloat)
    
test_df_new = test_df
for feature_idx in numeric_features[5:]:
    feature_id = train_df.columns[feature_idx]
    test_df_new.loc[:, feature_id] = test_df.loc[:, feature_id].map(toFloat)

In [264]:
train_df_new.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,0424,100005,100006,100007,10002,...,3193,32,320,33,34,37,38,39,669001,809009
count,38191.000000,38190.000000,38158.000000,38199.000000,38199.000000,30245.000000,12899.000000,17473.000000,15043.000000,20109.000000,...,36572.000000,17553.000000,16938.000000,17409.000000,13957.000000,17712.000000,17712.000000,14101.000000,5289.000000,4858.000000
mean,126.052918,77.023619,1.612536,1.406683,2.769719,72.601377,18.969464,14.448569,0.687269,9.183461,...,1.020535,3.768755,9.410126,2.076115,0.426795,57.697587,30.840653,7.730919,3.513815,1.620080
std,19.265082,13.385752,1.335518,0.341184,0.852217,9.175251,12.073726,2.659313,4.187223,4.083423,...,0.005984,1.258986,1.791910,0.737653,0.209125,13.335674,10.749796,6.360497,3.364076,0.803286
min,69.000000,37.000000,0.100000,0.150000,-1.220000,0.640000,4.000000,0.000000,0.000000,0.140000,...,1.003000,0.430000,0.000000,0.200000,0.090000,0.310000,0.120000,0.010000,0.010000,0.370000
25%,112.000000,68.000000,0.880000,1.160000,2.180000,66.000000,12.100000,12.300000,0.159000,6.400000,...,1.015000,2.900000,8.400000,1.600000,0.300000,53.500000,26.500000,4.800000,2.020000,1.400000
50%,124.000000,76.000000,1.270000,1.350000,2.690000,72.000000,13.000000,15.200000,0.201000,8.500000,...,1.020000,3.600000,9.500000,2.000000,0.400000,59.300000,32.100000,6.700000,3.090000,1.480000
75%,137.000000,85.000000,1.900000,1.600000,3.260000,78.000000,15.000000,16.200000,0.259000,11.130000,...,1.025000,4.400000,10.500000,2.400000,0.500000,65.000000,37.500000,8.700000,4.480000,1.570000
max,252.000000,974.000000,28.800000,4.780000,11.540000,148.000000,56.800000,32.200000,51.800000,52.100000,...,1.030000,18.300000,20.100000,15.600000,4.100000,92.200000,85.800000,55.800000,132.246000,21.590000


## 简单模型

In [269]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

feature = train_df_new.describe().columns.values.tolist()[5:]
label = train_df_new.describe().columns.values.tolist()[0:5]
X_train = train_df_new.loc[:,feature].fillna(0)
Y_train = train_df_new.loc[:,label].fillna(0)
X_test = test_df_new.loc[:,feature].fillna(0)
X_train.shape, Y_train.shape, X_test.shape

((38199, 72), (38199, 5), (9538, 72))

### 决策树

In [270]:
regr = DecisionTreeRegressor()
regr.fit(X_train, Y_train)
acc_decision_tree = round(regr.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [271]:
Y_pred_regr = regr.predict(X_test)
Y_pred_regr_df = pd.DataFrame(Y_pred_regr, columns=label)
Y_pred_regr_df.head()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
0,130.0,88.0,1.46,1.61,2.30
1,139.0,83.0,1.09,1.38,1.99
2,157.0,93.0,2.27,1.39,3.71
3,106.0,60.0,0.65,1.47,1.98
4,127.0,88.0,2.10,1.80,3.72


### 随机森林

In [274]:
from sklearn.cross_validation import train_test_split
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)

random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train1, Y_train1)
Y_pred = random_forest.predict(X_test1)
random_forest.score(X_train1, Y_train1)
acc_random_forest = round(random_forest.score(X_train1, Y_train1) * 100, 2)
acc_random_forest

89.17